In [168]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

import sklearn
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn.tree import plot_tree
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV


In [171]:
base_mod = pd.read_csv('https://raw.githubusercontent.com/MadavR/MIAD-VyS/main/EXA_2022_1_Todos_Limp.csv', sep=',')

In [140]:
base_mod.head()

,Unnamed: 0,ESTU_CONSECUTIVO,ESTU_GRADO,ESTU_GENERO,COLE_NOM_ESTABLECIMIENTO,COLE_COD_ICFES,COLE_NATURALEZA,COLE_CALENDARIO,COLE_JORNADA,COLE_COD_MCPIO,COLE_NOM_MCPIO,COLE_COD_DPTO,COLE_NOM_DPTO,EXA_MODALIDAD,EXA_N_RTAS_CORR_CN,EXA_N_RTAS_CORR_CC,EXA_N_RTAS_CORR_LC,EXA_N_RTAS_CORR_MT
0,0,22857330,3,M,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,13110,O,A,TARDE,91001,LETICIA,91,AMAZONAS,PA,0,0,8,0
1,1,22857841,3,F,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,13110,O,A,TARDE,91001,LETICIA,91,AMAZONAS,PA,0,0,6,10
2,2,22861931,3,F,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,13110,O,A,TARDE,91001,LETICIA,91,AMAZONAS,PA,0,0,8,0
3,3,22884935,3,F,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,13110,O,A,TARDE,91001,LETICIA,91,AMAZONAS,PA,0,0,7,10
4,4,22884936,3,M,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,13110,O,A,TARDE,91001,LETICIA,91,AMAZONAS,PA,0,0,14,12


In [141]:
base_mod.columns

Index(['Unnamed: 0', 'ESTU_CONSECUTIVO', 'ESTU_GRADO', 'ESTU_GENERO',
       'COLE_NOM_ESTABLECIMIENTO', 'COLE_COD_ICFES', 'COLE_NATURALEZA',
       'COLE_CALENDARIO', 'COLE_JORNADA', 'COLE_COD_MCPIO', 'COLE_NOM_MCPIO',
       'COLE_COD_DPTO', 'COLE_NOM_DPTO', 'EXA_MODALIDAD', 'EXA_N_RTAS_CORR_CN',
       'EXA_N_RTAS_CORR_CC', 'EXA_N_RTAS_CORR_LC', 'EXA_N_RTAS_CORR_MT'],
      dtype='object')

In [142]:
# En el escenario 5 consideramos todas las variables categóricas y numéricas
# Además, consideramos la categorización de los resultados: 0 (o-4 respuestas correctas), 1 (5-9 respuestas correctas),
# 2 (10-14 respuestas correctas) y 3 (15-20 respuestas correctas)

In [143]:
base_mod_cat = base_mod.assign(EXA_C_RTAS_CORR_CN = lambda x: ((base_mod['EXA_N_RTAS_CORR_CN'] // 5)))
base_mod_cat = base_mod_cat.assign(EXA_C_RTAS_CORR_CC = lambda x: ((base_mod['EXA_N_RTAS_CORR_CC'] // 5)))
base_mod_cat = base_mod_cat.assign(EXA_C_RTAS_CORR_LC = lambda x: ((base_mod['EXA_N_RTAS_CORR_LC'] // 5)))
base_mod_cat = base_mod_cat.assign(EXA_C_RTAS_CORR_MT = lambda x: ((base_mod['EXA_N_RTAS_CORR_MT'] // 5)))

In [144]:
base_mod_cat.head()

,Unnamed: 0,ESTU_CONSECUTIVO,ESTU_GRADO,ESTU_GENERO,COLE_NOM_ESTABLECIMIENTO,COLE_COD_ICFES,COLE_NATURALEZA,COLE_CALENDARIO,COLE_JORNADA,COLE_COD_MCPIO,...,COLE_NOM_DPTO,EXA_MODALIDAD,EXA_N_RTAS_CORR_CN,EXA_N_RTAS_CORR_CC,EXA_N_RTAS_CORR_LC,EXA_N_RTAS_CORR_MT,EXA_C_RTAS_CORR_CN,EXA_C_RTAS_CORR_CC,EXA_C_RTAS_CORR_LC,EXA_C_RTAS_CORR_MT
0,0,22857330,3,M,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,13110,O,A,TARDE,91001,...,AMAZONAS,PA,0,0,8,0,0,0,1,0
1,1,22857841,3,F,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,13110,O,A,TARDE,91001,...,AMAZONAS,PA,0,0,6,10,0,0,1,2
2,2,22861931,3,F,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,13110,O,A,TARDE,91001,...,AMAZONAS,PA,0,0,8,0,0,0,1,0
3,3,22884935,3,F,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,13110,O,A,TARDE,91001,...,AMAZONAS,PA,0,0,7,10,0,0,1,2
4,4,22884936,3,M,INSTITUCION EDUCATIVA SAGRADO CORAZON DE JESUS,13110,O,A,TARDE,91001,...,AMAZONAS,PA,0,0,14,12,0,0,2,2


In [145]:
base_mod_cat.columns

Index(['Unnamed: 0', 'ESTU_CONSECUTIVO', 'ESTU_GRADO', 'ESTU_GENERO',
       'COLE_NOM_ESTABLECIMIENTO', 'COLE_COD_ICFES', 'COLE_NATURALEZA',
       'COLE_CALENDARIO', 'COLE_JORNADA', 'COLE_COD_MCPIO', 'COLE_NOM_MCPIO',
       'COLE_COD_DPTO', 'COLE_NOM_DPTO', 'EXA_MODALIDAD', 'EXA_N_RTAS_CORR_CN',
       'EXA_N_RTAS_CORR_CC', 'EXA_N_RTAS_CORR_LC', 'EXA_N_RTAS_CORR_MT',
       'EXA_C_RTAS_CORR_CN', 'EXA_C_RTAS_CORR_CC', 'EXA_C_RTAS_CORR_LC',
       'EXA_C_RTAS_CORR_MT'],
      dtype='object')

In [146]:
base_mod_cat.ESTU_GRADO.value_counts()

6     2645
5     2286
4     2256
7     2099
3     2073
8     1961
9     1699
10    1478
11    1041
Name: ESTU_GRADO, dtype: int64

In [147]:
base_mod_cat.ESTU_GENERO.value_counts()

F    8790
M    8748
Name: ESTU_GENERO, dtype: int64

In [148]:
base_mod_cat.COLE_NATURALEZA.value_counts()

O     16688
NO      850
Name: COLE_NATURALEZA, dtype: int64

In [149]:
base_mod_cat.COLE_CALENDARIO.value_counts()

A    17538
Name: COLE_CALENDARIO, dtype: int64

In [150]:
base_mod_cat.COLE_JORNADA.value_counts()

MAÑANA      11972
UNICA        2288
COMPLETA     2248
TARDE        1030
Name: COLE_JORNADA, dtype: int64

In [151]:
base_mod_cat.EXA_MODALIDAD.value_counts()

PA     12894
ON      2721
OFF     1768
PAL      155
Name: EXA_MODALIDAD, dtype: int64

In [152]:
base_mod_cat.EXA_C_RTAS_CORR_CC.value_counts()

0    8467
1    6525
2    2140
3     353
4      53
Name: EXA_C_RTAS_CORR_CC, dtype: int64

In [153]:
base_mod_cat.EXA_C_RTAS_CORR_CN.value_counts()

0    8150
1    6375
2    2415
3     543
4      55
Name: EXA_C_RTAS_CORR_CN, dtype: int64

In [154]:
base_mod_cat.EXA_C_RTAS_CORR_LC.value_counts()

1    8662
2    5061
0    2269
3    1512
4      34
Name: EXA_C_RTAS_CORR_LC, dtype: int64

In [155]:
base_mod_cat.EXA_C_RTAS_CORR_MT.value_counts()

1    9711
0    4756
2    2651
3     390
4      30
Name: EXA_C_RTAS_CORR_MT, dtype: int64

In [156]:
# Crea una lista de las variables categóricas
categoricas_var = ["ESTU_GENERO", "COLE_NATURALEZA", "COLE_JORNADA", "EXA_MODALIDAD"]

# Realiza la codificación one hot
for var in categoricas_var:
    base_mod_cat = pd.get_dummies(base_mod_cat, columns=[var])

In [157]:
base_mod_cat['EXA_C_RTAS_CORR_CN'] = base_mod_cat['EXA_C_RTAS_CORR_CN'].replace([4], 3)
base_mod_cat['EXA_C_RTAS_CORR_CC'] = base_mod_cat['EXA_C_RTAS_CORR_CC'].replace([4], 3)
base_mod_cat['EXA_C_RTAS_CORR_LC'] = base_mod_cat['EXA_C_RTAS_CORR_LC'].replace([4], 3)
base_mod_cat['EXA_C_RTAS_CORR_MT'] = base_mod_cat['EXA_C_RTAS_CORR_MT'].replace([4], 3)

In [158]:
base_mod_cat.EXA_C_RTAS_CORR_CC.value_counts()

0    8467
1    6525
2    2140
3     406
Name: EXA_C_RTAS_CORR_CC, dtype: int64

In [159]:
base_mod_cat.EXA_C_RTAS_CORR_CN.value_counts()

0    8150
1    6375
2    2415
3     598
Name: EXA_C_RTAS_CORR_CN, dtype: int64

In [160]:
base_mod_cat.EXA_C_RTAS_CORR_LC.value_counts()

1    8662
2    5061
0    2269
3    1546
Name: EXA_C_RTAS_CORR_LC, dtype: int64

In [161]:
base_mod_cat.EXA_C_RTAS_CORR_MT.value_counts()

1    9711
0    4756
2    2651
3     420
Name: EXA_C_RTAS_CORR_MT, dtype: int64

In [172]:
base_mod_cat.columns

Index(['Unnamed: 0', 'ESTU_CONSECUTIVO', 'ESTU_GRADO',
       'COLE_NOM_ESTABLECIMIENTO', 'COLE_COD_ICFES', 'COLE_CALENDARIO',
       'COLE_COD_MCPIO', 'COLE_NOM_MCPIO', 'COLE_COD_DPTO', 'COLE_NOM_DPTO',
       'EXA_N_RTAS_CORR_CN', 'EXA_N_RTAS_CORR_CC', 'EXA_N_RTAS_CORR_LC',
       'EXA_N_RTAS_CORR_MT', 'EXA_C_RTAS_CORR_CN', 'EXA_C_RTAS_CORR_CC',
       'EXA_C_RTAS_CORR_LC', 'EXA_C_RTAS_CORR_MT', 'ESTU_GENERO_F',
       'ESTU_GENERO_M', 'COLE_NATURALEZA_NO', 'COLE_NATURALEZA_O',
       'COLE_JORNADA_COMPLETA', 'COLE_JORNADA_MAÑANA', 'COLE_JORNADA_TARDE',
       'COLE_JORNADA_UNICA', 'EXA_MODALIDAD_OFF', 'EXA_MODALIDAD_ON',
       'EXA_MODALIDAD_PA', 'EXA_MODALIDAD_PAL'],
      dtype='object')

In [163]:
X = base_mod_cat[['ESTU_GRADO',  'COLE_COD_ICFES', 'COLE_COD_MCPIO', 'COLE_COD_DPTO', 'EXA_C_RTAS_CORR_CN', 'EXA_C_RTAS_CORR_CC', 'EXA_C_RTAS_CORR_MT',
               'ESTU_GENERO_F', 'ESTU_GENERO_M', 'COLE_NATURALEZA_NO', 'COLE_NATURALEZA_O',
               'COLE_JORNADA_COMPLETA', 'COLE_JORNADA_MAÑANA', 'COLE_JORNADA_TARDE', 'COLE_JORNADA_UNICA',
              'EXA_MODALIDAD_OFF', 'EXA_MODALIDAD_ON', 'EXA_MODALIDAD_PA', 'EXA_MODALIDAD_PAL']]

# Selecciona la variable dependiente
y = base_mod_cat['EXA_C_RTAS_CORR_LC']

In [173]:
X = base_mod_cat[['ESTU_GRADO',  'COLE_COD_ICFES', 'COLE_COD_MCPIO', 'COLE_COD_DPTO',
                    'EXA_C_RTAS_CORR_CN', 'EXA_C_RTAS_CORR_CC', 'EXA_C_RTAS_CORR_MT',
               'ESTU_GENERO_F', 'COLE_NATURALEZA_O',
               'COLE_JORNADA_COMPLETA', 'COLE_JORNADA_MAÑANA', 'COLE_JORNADA_TARDE',
                  'EXA_MODALIDAD_OFF', 'EXA_MODALIDAD_ON', 'EXA_MODALIDAD_PA']]

# Selecciona la variable dependiente
y = base_mod_cat['EXA_C_RTAS_CORR_LC']

In [174]:
# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [175]:
# Entrena el modelo de bosque aleatorio
model_RF= RandomForestClassifier(n_estimators=200, max_depth=80)
model_RF.fit(X_train, y_train)

# Evalúa el modelo
score_RF = model_RF.score(X_test, y_test)
print(score_RF)

prediccion_RF = model_RF.predict(X_test)
MSE_RF = np.average(np.square(prediccion_RF - y_test))
print("El MSE del modelo en la muestra de evaluación es igual a:")
print(MSE_RF)

0.5353477765108324
El MSE del modelo en la muestra de evaluación es igual a:
0.6348346636259977


In [176]:
# Entrenar el algoritmo de Gradient Boosting
model_GB = GradientBoostingClassifier(n_estimators=100, learning_rate=0.3)
model_GB.fit(X_train, y_train)

# Evalúa el modelo
score_GB = model_GB.score(X_test, y_test)
print(score_GB)

prediccion_GB = model_GB.predict(X_test)
MSE_GB = np.average(np.square(prediccion_GB - y_test))
print("El MSE del modelo en la muestra de evaluación es igual a:")
print(MSE_GB)

0.57212086659065
El MSE del modelo en la muestra de evaluación es igual a:
0.5316419612314709


In [177]:
# Entrenar el algoritmo de XGBoost
model_XGB = XGBClassifier(n_estimators=100, learning_rate=0.2)
model_XGB.fit(X_train, y_train)

# Evalúa el modelo
score_XGB = model_XGB.score(X_test, y_test)
print(score_XGB)

prediccion_XGB = model_XGB.predict(X_test)
MSE_XGB = np.average(np.square(prediccion_XGB - y_test))
print("El MSE del modelo en la muestra de evaluación es igual a:")
print(MSE_XGB)



0.5763968072976055
El MSE del modelo en la muestra de evaluación es igual a:
0.5248004561003421


In [178]:
# Entrenar el algoritmo de XGBoost
model_XGBop = XGBClassifier(n_estimators=200, learning_rate=0.2, max_depth=5)
model_XGBop.fit(X_train, y_train)

# Evalúa el modelo
score_XGBop = model_XGBop.score(X_test, y_test)
print(score_XGBop)

prediccion_XGBop = model_XGBop.predict(X_test)
MSE_XGBop = np.average(np.square(prediccion_XGBop - y_test))
print("El MSE del modelo en la muestra de evaluación es igual a:")
print(MSE_XGBop)

0.5812428734321551
El MSE del modelo en la muestra de evaluación es igual a:
0.5219498289623717


In [179]:
# Definir el modelo a optimizar
model = XGBClassifier()

# Definir el espacio de parámetros a explorar
param_grid = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.1, 0.2, 0.3],
    "max_depth": [3, 5, 7],
}

# Instanciar el GridSearchCV
gscv = GridSearchCV(model, param_grid, scoring="accuracy", n_jobs=-1)

# Entrenar el modelo
gscv.fit(X_train, y_train)

# Imprimir los resultados
print(gscv.best_params_)
print(gscv.best_score_)

{'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 300}
0.5762651461154669


In [180]:
base_mod_cat.to_csv('EXA_2022_1_Todos_Limp_Cat_1.csv')